In [0]:
!pip install rake_nltk

In [0]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 95.4MB 39.0MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-cofhkw8h/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
!python -m spacy link en_core_web_md enmd

✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_md -->
/usr/local/lib/python3.6/dist-packages/spacy/data/enmd
You can now load the model via spacy.load('enmd')


In [0]:
!pip install textacy

     |████████████████████████████████| 163kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 38.1MB/s 
     |████████████████████████████████| 450kB 40.2MB/s 
     |████████████████████████████████| 51kB 14.7MB/s 
     |████████████████████████████████| 5.4MB 33.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/88/f3/11/9817b001e59ab04889e8cffcbd9087e2e2155b9ebecfc8dd38
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built cytoolz python-levenshtein
  Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2


In [0]:
import pandas as pd
from rake_nltk import Metric, Rake
from nltk.tree import Tree
from nltk.corpus import stopwords, wordnet
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import textacy
import spacy

In [0]:
# data = pd.read_csv("testAPINew_ID.csv")
data = pd.read_csv("Test.csv")

In [0]:
import spacy
text = """
Apple is looking at buying U.K. startup for $1 billion, 1) * hi guitar (
open ).) diference
regardless )
). base
). putte
gave (
+ 2db
germany ).
) harmonic
obvious ;)
hint )-
) none
00 ).
( plus
key )
c #)
) put
[ update
reliable (
deoxit (
amazon )
( especially
clone ).
got ).
know )
others ).
?&# 34
later ).') 
"""
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [0]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


 
 SPACE _SP  
 False False
Apple Apple PROPN NNP nsubj Xxxxx True False
is be VERB VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False
, , PUNCT , punct , False False
1 1 NUM CD npadvmod d False False
) ) PUNCT -RRB- punct ) False False
* * PUNCT NFP ROOT * False False
hi hi INTJ UH intj xx True False
guitar guitar NOUN NN ROOT xxxx True False
( ( PUNCT -LRB- punct ( False False

 
 SPACE _SP  
 False False
open open ADJ JJ amod xxxx True False
) ) PUNCT -RRB- punct ) False False
. . PUNCT . punct . False False
) ) PUNCT -RRB- punct ) False False
diference diference NOUN NN ROOT xxxx True False

 
 SPACE _SP  
 False False
regardless regardless ADV RB advmod xxxx True 

In [0]:
for token in doc:
  if token.pos_ not in ["PUNCT", "NUM", "SYM" ]:
    print (token.text)



Apple
is
looking
at
buying
U.K.
startup
for
hi
guitar


open
diference


regardless


base


putte


gave


2db


germany


harmonic


obvious
;)


hint


none




plus


key


c
#


put


update


reliable


deoxit


amazon


especially


clone


got


know


others


&


later




In [0]:
def getLemma(eachValue):
  try:
    doc = nlp(eachValue)
    LemmatizedSentence = []
    for token in doc:2
      if token.pos_ not in ["PUNCT", "NUM", "SYM" ]:
        if token.lemma_ == "-PRON-":
          LemmatizedSentence.append(token.text)
        else:
          LemmatizedSentence.append(token.lemma_)
    return " ".join(LemmatizedSentence)
  except TypeError:
    return " "
  
global nlp
nlp = spacy.load("enmd")
%time data['UpdatedReviewText'] = data.reviewText.apply(getLemma)

CPU times: user 1min 59s, sys: 452 ms, total: 1min 59s
Wall time: 1min 59s


In [0]:
data.to_csv("Test.csv")

In [0]:
def get_stopword_lists():
    '''
    params: 
          None
          
    returns:
          a. sw_with_custom : nltk stopwords extended with stopwords custom defined by me. 
                              This will be used in text_preprocess() and RAKE algorithms
          
          b. sw_final: All stopwords including custom sw and punctuation marks.
                       This will be used in our Vectorizer.
    '''
    #make different stopwords lists which will be filtered out in text_preprocess() and which will be used for Rake

    sw = stopwords.words("english")
    
    # StopWords to ADD:
    # 1),2)  and ")"
    #define some custom stopwords
    custom_sw = ['since', 'because', '"',"'","-",'...', '..','``', "''" ,':','*','**','***','****','*****','upon','would','should','could','can'
                 ,'has','bar','good','really','bad']
    sw_with_custom = sw.copy()
    sw_with_custom.extend(custom_sw)
    
    # punctuations are stopwords too
    sw_with_punc = sw.copy()
    sw_with_punc.extend(list(string.punctuation))
    sw_with_custom_punc = sw_with_custom.copy()
    sw_with_custom_punc.extend(sw_with_punc)
    sw_with_custom_punc = list(set(sw_with_custom_punc))
    
    # define list of words to remove from nltk defined stopword list. Excess words and negation words are removed.
    #to_remove_from_sw =['no', 'not', 'nor', 'so', 'too', 'very']
    #sw = list(set(sw)-set(to_remove_from_sw))
    
    #add titlecase versions of all stopwords
    sw_final = sw_with_custom_punc.copy()
    sw_final.extend(list(map(lambda x: x.title(), sw_with_custom_punc)))
    sw_final = list(set(sw_final))
    
    return sw_with_custom, sw_final

In [0]:
def getRakeTopic(eachvalue):
  try:
    r = Rake(stopwords = sw_with_custom, ranking_metric=Metric.WORD_DEGREE, min_length=2,max_length=2, punctuations=["'",'.','?',';','!','..','?!','!?',',','*,','**,','***,','****,','*****,']) # Uses stopwords for english from NLTK, and all puntuation characters.
    r.extract_keywords_from_text(eachvalue)
    return r.get_ranked_phrases()
  except TypeError:
    return []

In [0]:
global sw_with_custom
sw_with_custom,sw_final = get_stopword_lists()
data['Topic'] = data.UpdatedReviewText.apply(getRakeTopic)

In [0]:
import math
def getNtopElementsinList(eachValue):
  if len(eachValue) > 1:
    return eachValue[:math.ceil(Fraction*len(eachValue))]
  else:
    return eachValue
  
def getNBottomElementsinList(eachValue):
  if len(eachValue) > 1:
    return eachValue[-math.floor(Fraction*len(eachValue)):]
  else:
    return []

global Fraction
Fraction = 0.5
data['Top50%Topic'] = data.Topic.apply(getNtopElementsinList)
data['Botton50%Topic'] = data.Topic.apply(getNBottomElementsinList)

In [0]:
def getSubjectVerbObject(eachValue):
  try:
    return list(textacy.extract.subject_verb_object_triples(nlp(eachValue)))
  except TypeError:
    return []
import spacy
global nlp
nlp = spacy.load("enmd")
# data['SubjectVerbObject'] = data.reviewText.apply(getSubjectVerbObject)

In [0]:
data[['Topic','ID','Top50%Topic','Botton50%Topic','SubjectVerbObject']]

,Topic,ID,Top50%Topic,Botton50%Topic,SubjectVerbObject
0,"[well buy, pop sound, honestly work]",1068606,"[well buy, pop sound]",[honestly work],"[(it, 's supposed, to), (it, is, one)]"
1,"[stop put, small hint, screen carry, record di...",15241064,"[stop put, small hint, screen carry, record di...","[pop filter, expensive one, even well, double ...","[(one, carries, hint), (I, used, to buy), (I, ..."
2,"[primary job, popping sound, otherwise produce...",4889797,"[primary job, popping sound, otherwise produce]","[noticeable reduction, little coax, high frequ...","[(job, is, to block), (that, produce, popping ..."
3,"[prevent pop, mxl mic, marginally able, avoid ...",12696788,"[prevent pop, mxl mic]","[marginally able, avoid sag]","[(windscreen, protects, MXL), (windscreen, pro..."
4,"[studio filter, record vocal, pop filter, get ...",31343259,"[studio filter, record vocal, pop filter]","[get record, perform like]","[(you, 're recording, vocals), (this, will eli..."
5,"[read instruction, heavy cord]",37344394,[read instruction],[heavy cord],"[(I, bought, one), (I, learned, to coil), (I, ..."
6,[lifetime warranty],33402836,[lifetime warranty],[],"[(I, have used, monster cables)]"
7,"[pay back, money back, go back, think waves, p...",16698218,"[pay back, money back, go back, think waves, p...","[pedal board, plus year, little disappointed, ...","[(I, use, cable), (I, use, to run), (I, bought..."
8,"[year get, well construct, degree plug]",44819157,"[year get, well construct]",[degree plug],"[(I, have, several), (I, wanted, degree plug)]"
9,"[monster make, monster cables, work great, lin...",1741598,"[monster make, monster cables, work great, lin...","[lifetime warranty, hurt either, go wrong]","[(This, is, top)]"


In [0]:
def getUniqueTopicsFrequency(DfColumn_Topics):
  AllTopicList = [item for sublist in list(DfColumn_Topics) for item in sublist]
#   AllNPList = [item for sublist in list(DfColumn_NP) for item in sublist]
  UniqueTopicList = list(set(AllTopicList))
#   UniqueNPList = list(set(AllNPList))
  CombinedList = AllTopicList
  TopicFrequency = collections.Counter(CombinedList)
  print(TopicFrequency)
  df = pd.DataFrame.from_dict(TopicFrequency, orient='index').reset_index()
#   df['Topic'] = df.index
  df.columns = df.columns.map(str)
  df['Strength%'] = (df['0']/len(CombinedList))*100
  df.rename(columns = {'0':'TotalCountAllAnswer', 'index':'ImportantTopic'}, inplace=True)
#   df['Strength%'] = (df[0]/TotalUniqueTopic)*100   ## Divide by total Answers 
  return df.sort_values(by=['TotalCountAllAnswer'], ascending=False),UniqueTopicList 

import collections
ImportantTopicsDF, TopicList = getUniqueTopicsFrequency(data['Botton50%Topic'] )                                                                                                 
 

Counter({'highly recommend': 68, 'acoustic guitar': 47, 'long time': 46, 'go wrong': 45, 'electric guitar': 39, 'make sure': 37, 'feel like': 28, 'little bit': 24, 'big deal': 23, 'last longer': 22, 'great product': 22, 'great price': 22, 'build quality': 21, 'go back': 21, 'addario string': 21, 'also use': 20, 'first time': 19, 'gig bag': 19, 'change string': 19, 'many year': 18, '& 34': 18, 'ever use': 18, 'ernie ball': 17, 'guitar case': 17, 'les paul': 16, 'build like': 16, 'much well': 15, 'definitely recommend': 15, 'get rid': 14, 'great value': 14, 'low price': 13, 'anything else': 13, 'look like': 13, 'last forever': 13, 'first thing': 12, 'even though': 12, 'mic stand': 12, 'sound great': 12, 'play guitar': 12, 'home studio': 11, 'light weight': 11, 'bottom line': 11, 'much easy': 11, 'go ahead': 11, 'classic rock': 11, 'also like': 11, 'high quality': 11, 'get use': 11, 'guitar center': 10, 'buy another': 10, 'absolutely love': 10, 'may want': 10, '&# 34': 10, 'great way': 10

In [0]:
for topic in TopicListSubset:
  if topic in "long time No See":
    print ("Lavi")

Lavi


In [0]:
ImportantTopicsDF['ImportantTopic'][0]

'honestly work'

In [0]:
def testfunc(x,df1,df2, TopicList):
  UpdateIndex =[]
  for Topic in TopicList:
    if Topic in df1['UpdatedReviewText'][x]:
      UpdateIndex.append(df1['ID'][x])
   
  return UpdateIndex


# #   if df1['UpdatedReviewText'][x].contains(df2['ImportantTopic'][x]):
#   if df2['ImportantTopic'][x] in df1['UpdatedReviewText'][x]:
#     return df1['ID'][x]
#   else:
#     return 0

def testfunc2(x,data,df2):
  String2Match = ImportantTopicsDF['ImportantTopic'][x]
#   print(String2Match)
  try:
    L = data[data['UpdatedReviewText'].str.contains(String2Match, na = False)]['ID'].tolist()
    return L
  except:
    print(String2Match)
    return []




In [0]:
ImportantTopicsDF.reset_index(inplace=True)

In [0]:

ImportantTopicsDF['Flag'] = ImportantTopicsDF.index.map(lambda x: testfunc2(x, data, ImportantTopicsDF))
ImportantTopicsDF
# data['Flag'] = data.index.map(lambda x: testfunc(x,data,ImportantTopicsDF,TopicListSubset))

guitar (
open ).
) diference
regardless )
). base
). putte
gave (
+ 2db
germany ).
) harmonic
obvious ;)
hint )-
) none
00 ).
( plus
key )
c #)
) put
[ update
reliable (
deoxit (
amazon )
( especially
clone ).
got ).
know )
others ).
?&# 34
later ).


,ImportantTopic,TotalCountAllAnswer,Strength%,Flag
44,highly recommend,68,0.510319,"[13675076, 21166435, 2158182, 25974220, 726454..."
1883,acoustic guitar,47,0.352720,"[341383, 28097024, 551487, 34349528, 27552546,..."
379,long time,46,0.345216,"[6056122, 10445289, 43098922, 8613957, 4129554..."
23,go wrong,45,0.337711,"[1741598, 38998254, 41532205, 28983748, 161481..."
82,electric guitar,39,0.292683,"[23525998, 5842487, 1952125, 3373026, 23722939..."
63,make sure,37,0.277674,"[33701286, 43098922, 21431925, 8478166, 436187..."
220,feel like,28,0.210131,"[21166435, 25435820, 46640146, 27361351, 47212..."
129,little bit,24,0.180113,"[41486151, 12221401, 41646436, 43759657, 64992..."
775,big deal,23,0.172608,"[43618799, 2469942, 17226580, 4223783, 1750639..."
1524,great product,22,0.165103,"[26432743, 44816838, 39521342, 24051026, 24188..."


In [0]:
ImportantTopicsDF.head(30)


,ImportantTopic,TotalCountAllAnswer,Strength%,Flag
44,highly recommend,68,0.510319,"[13675076, 21166435, 2158182, 25974220, 726454..."
1883,acoustic guitar,47,0.352720,"[341383, 28097024, 551487, 34349528, 27552546,..."
379,long time,46,0.345216,"[6056122, 10445289, 43098922, 8613957, 4129554..."
23,go wrong,45,0.337711,"[1741598, 38998254, 41532205, 28983748, 161481..."
82,electric guitar,39,0.292683,"[23525998, 5842487, 1952125, 3373026, 23722939..."
63,make sure,37,0.277674,"[33701286, 43098922, 21431925, 8478166, 436187..."
220,feel like,28,0.210131,"[21166435, 25435820, 46640146, 27361351, 47212..."
129,little bit,24,0.180113,"[41486151, 12221401, 41646436, 43759657, 64992..."
775,big deal,23,0.172608,"[43618799, 2469942, 17226580, 4223783, 1750639..."
1524,great product,22,0.165103,"[26432743, 44816838, 39521342, 24051026, 24188..."


In [0]:
data[data['reviewText'].str.contains("buy one", na = False)]['ID'].tolist()

[1060744,
 43759657,
 20856924,
 23793016,
 43618799,
 8635818,
 44316511,
 30220360,
 46081161,
 30854600,
 32270047,
 24993110,
 10005471,
 31418084,
 12765478,
 41511868,
 11103821,
 38541579]